In [55]:
# Import the dependencies
from pymongo import MongoClient
from pprint import pprint
import json

In [56]:
# Make a connection to the MongoDB server
mongo = MongoClient(port=27017)

In [57]:
# List the databases
print(mongo.list_database_names())

['admin', 'config', 'epa', 'flight_data', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [58]:
# Connect to the 'flight_data' database
db = mongo['flight_data']

Import the data from "US Airline Flight Routes and Fares 1993-2024.csv" using this command:

```shell
mongoimport --db flight_data --collection flights --drop --type csv --headerline --file "../Data/CSV/cleaned_flight_data.csv"
```

In [59]:
# List the collections
print(db.list_collection_names())

['flights']


In [60]:
# Return the first document in the 'flights' collection
pprint(db.flights.find_one())

{'Year': 2010,
 '_id': ObjectId('679c2bfb9f914d124e2d32f5'),
 'arrival_airport': 'TPA',
 'arrival_airport_id': 15304,
 'arrival_city': 'Tampa, FL (Metropolitan Area)',
 'arrival_city_id': 33195,
 'arrival_latitude': 37.8606,
 'arrival_longitude': -78.804199,
 'carrier_lg': 'Delta Air Lines Inc.',
 'carrier_low': 'US Airways Group Inc.',
 'departure_airport': 'SLC',
 'departure_airport_id': 14869,
 'departure_city': 'Salt Lake City, UT',
 'departure_city_id': 34614,
 'departure_latitude': 40.758478,
 'departure_longitude': -111.888142,
 'fare': 226.59,
 'fare_lg': 247.69,
 'fare_low': 166.99,
 'large_ms': 0.38,
 'lf_ms': 0.2,
 'nsmiles': 1887,
 'passengers': 200,
 'quarter': 1,
 'tbl': 'Table 1a',
 'tbl1apk': '201011486915304SLCTPA'}


In [61]:
# Add a new collection to the database using this document for reference
db.create_collection('locations')

# Iterate through each document in the 'flights' collection and add it to the 'locations' collection
for flight in db.flights.find():
    location_document = {
        "_id": flight["_id"],
        "arrival_airport": flight["arrival_airport"],
        "arrival_airport_id": flight["arrival_airport_id"],
        "arrival_city": flight["arrival_city"],
        "arrival_city_id": flight["arrival_city_id"],
        "arrival_latitude": flight["arrival_latitude"],
        "arrival_longitude": flight["arrival_longitude"],
        "departure_airport": flight["departure_airport"],
        "departure_airport_id": flight["departure_airport_id"],
        "departure_city": flight["departure_city"],
        "departure_city_id": flight["departure_city_id"],
        "departure_latitude": flight["departure_latitude"],
        "departure_longitude": flight["departure_longitude"]
    }
    db.locations.insert_one(location_document)

In [62]:
# Print the first document in the "locations" collection
pprint(db.locations.find_one())

{'_id': ObjectId('679c2bfb9f914d124e2d32f5'),
 'arrival_airport': 'TPA',
 'arrival_airport_id': 15304,
 'arrival_city': 'Tampa, FL (Metropolitan Area)',
 'arrival_city_id': 33195,
 'arrival_latitude': 37.8606,
 'arrival_longitude': -78.804199,
 'departure_airport': 'SLC',
 'departure_airport_id': 14869,
 'departure_city': 'Salt Lake City, UT',
 'departure_city_id': 34614,
 'departure_latitude': 40.758478,
 'departure_longitude': -111.888142}


In [63]:
from bson import ObjectId

# Write the "flights" collection to a json file
def convert_objectid(obj):
    if isinstance(obj, ObjectId):
        return str(obj)
    raise TypeError("Object of type %s is not JSON serializable" % type(obj).__name__)

with open('../Data/JSON/flights.json', 'w') as f:
    json.dump(list(db.flights.find()), f, default=convert_objectid)

In [64]:
# Write the "locations" collection to a json file
with open('../Data/JSON/locations.json', 'w') as f:
    json.dump(list(db.locations.find()), f, default=convert_objectid)

In [65]:
# Close the connection
mongo.close()